In [12]:
# 1、获取数据
import pandas
facebook_data = pandas.read_csv("E:/python大数据资料\黑马-02-最简单快速入门python机器学习/资料/机器学习day2资料/02-代码/FBlocation/train.csv")

In [13]:
facebook_data.head()

,row_id,x,y,accuracy,time,place_id
0,0,0.7941,9.0809,54,470702,8523065625
1,1,5.9567,4.7968,13,186555,1757726713
2,2,8.3078,7.0407,74,322648,1137537235
3,3,7.3665,2.5165,65,704587,6567393236
4,4,4.0961,1.1307,31,472130,7440663949


In [14]:
# 2、数据处理
# （1）缩小数据范围（3<x<3.5,2<y<2.5）
facebook_data = facebook_data.query("x>3 & x<3.5 & y>2 & y<2.5")

In [15]:
facebook_data.head()

,row_id,x,y,accuracy,time,place_id
101,101,3.4777,2.0938,71,115352,3762957785
463,463,3.2228,2.4060,10,53671,9883722775
861,861,3.4123,2.1522,66,521528,3631406834
2255,2255,3.3950,2.3300,30,328984,5802159543
3260,3260,3.4463,2.1168,9,65262,1894108773


In [16]:
# (2)将时间戳转换成年月日时分秒的形式
date_data = pandas.to_datetime(facebook_data["time"],unit="s")

In [18]:
date = pandas.DatetimeIndex(date_data)

In [24]:
facebook_data["day"] = date.day
facebook_data["weekday"] = date.weekday
facebook_data["hour"] = date.hour

In [26]:
facebook_data.head()

,row_id,x,y,accuracy,time,place_id,day,weekday,hour
101,101,3.4777,2.0938,71,115352,3762957785,2,4,8
463,463,3.2228,2.4060,10,53671,9883722775,1,3,14
861,861,3.4123,2.1522,66,521528,3631406834,7,2,0
2255,2255,3.3950,2.3300,30,328984,5802159543,4,6,19
3260,3260,3.4463,2.1168,9,65262,1894108773,1,3,18


In [30]:
# (3)过滤签到次数少的地点
place_count = facebook_data.groupby("place_id").count()["row_id"]

In [33]:
place_count[place_count>3].head()

place_id
1006234733    57
1008823061     6
1012580558     8
1025585791    39
1031070090    70
Name: row_id, dtype: int64

In [41]:
facebook_data = facebook_data[facebook_data["place_id"].isin(place_count[place_count>3].index.values)]

In [42]:
facebook_data.head()

,row_id,x,y,accuracy,time,place_id,day,weekday,hour
101,101,3.4777,2.0938,71,115352,3762957785,2,4,8
463,463,3.2228,2.4060,10,53671,9883722775,1,3,14
861,861,3.4123,2.1522,66,521528,3631406834,7,2,0
2255,2255,3.3950,2.3300,30,328984,5802159543,4,6,19
3260,3260,3.4463,2.1168,9,65262,1894108773,1,3,18


In [46]:
# 筛选特征值(x)和目标值(y)
x = facebook_data[["x","y","accuracy","day","weekday","hour"]]
y = facebook_data["place_id"]

In [45]:
x.head()

,x,y,accuracy,day,weekday,hour
101,3.4777,2.0938,71,2,4,8
463,3.2228,2.4060,10,1,3,14
861,3.4123,2.1522,66,7,2,0
2255,3.3950,2.3300,30,4,6,19
3260,3.4463,2.1168,9,1,3,18


In [48]:
y.head()

101     3762957785
463     9883722775
861     3631406834
2255    5802159543
3260    1894108773
Name: place_id, dtype: int64

In [68]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

In [51]:
#  3、划分数据集
x_train,x_test,y_train,y_test = train_test_split(x,y)

In [64]:
# 4、特征工程(标准化)
trainsfer = StandardScaler()
x_train = trainsfer.fit_transform(x_train)
x_test = trainsfer.transform(x_test)

In [65]:
# 5、knn算法预估器
knn = KNeighborsClassifier()

In [71]:
# 6、添加网格搜索和交叉验证来进行模型调优
param_dict = {"n_neighbors":[3, 5, 7, 9]}
knn = GridSearchCV(knn,param_grid= param_dict,cv=3)

In [72]:
knn.fit(x_train,y_train)

D:\Program Files\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='warn', n_jobs=None, param_grid={'n_neighbors': [3, 5, 7, 9]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [73]:
# 5、模型评估
# 方法一：直接比对真实值和预估值
predict = knn.predict(x_test)
print("predict:\n",predict)
print("直接对比真实值和预估值:\n", y_test == predict)
# 方法二：计算准确率
score = knn.score(x_test,y_test)
print("准确率：",score)
# 最佳参数
print("最佳参数：",knn.best_params_)
# 最佳结果
print("最佳结果：",knn.best_score_)
# 最佳估计器
print("最佳估计器：",knn.best_estimator_)
# 交叉验证结果
print("交叉验证结果：",knn.cv_results_)

predict:
 [1362464377 4121420392 5335762826 ... 4476640394 4561558464 1025585791]
直接对比真实值和预估值:
 2517313      True
10882551    False
19705385    False
25673759    False
5945165      True
7895978     False
3841300      True
16347476    False
4416525      True
1189075     False
28246128    False
1840449     False
7233299     False
11229869     True
9257613     False
626618      False
2821348     False
27821024    False
19804879    False
9867808      True
17422354    False
23086707     True
8020336      True
20956560    False
19909947    False
15024591    False
2776297     False
20128845    False
25849689    False
26550061    False
            ...  
25457053    False
2572127      True
27317927     True
4225885      True
7310883     False
26806005    False
24672138    False
3104504     False
1331995      True
19846061    False
23326775    False
16997759    False
27974040    False
3929501     False
12895495    False
523783      False
4024024     False
13752824    False
9437646     False
2027